In [ ]:
!pip install huggingface_hub[cli]
!pip install utilsforecast
!pip install praxis
!pip install paxml
!pip install jax
!pip install einshape

In [ ]:
!pip install -q -e git+https://github.com/google-research/timesfm.git@master#egg=timesfm

In [ ]:
!pip install timesfm

In [ ]:
import timesfm

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0].


In [ ]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

torch_model.ckpt:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

In [ ]:
#Univariate

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

start_date = datetime.now()
num_days = 365
dates = [start_date + timedelta(days=i) for i in range(num_days)]
sales = [100 + i * 10 + np.random.normal(0, 10) for i in range(num_days)]

data = pd.DataFrame({
    'ds': dates,
    'sales': sales,
    'unique_id': "sales"
})

In [ ]:
#Multivariate (other variables get ignored)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

start_date = datetime.now()
num_days = 365
dates = [start_date + timedelta(days=i) for i in range(num_days)]
sales = [100 + i * 10 + np.random.normal(0, 10) for i in range(num_days)]
ad_spend = [50 + 20 * np.sin(2 * np.pi * i / 30) + np.random.normal(0, 5) for i in range(num_days)]
temperature = [20 + 10 * np.sin(2 * np.pi * i / 365) + np.random.normal(0, 2) for i in range(num_days)]

data = pd.DataFrame({
    'ds': dates,
    'sales': sales,
    'ad_spend': ad_spend,
    'temperature': temperature,
    'unique_id': "sales"
})

In [ ]:
%%time
forecast_df = tfm.forecast_on_df(
    inputs=data,
    freq="D",
    value_name="sales",
    num_jobs=-1,
)

Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
CPU times: user 44.1 s, sys: 241 ms, total: 44.4 s
Wall time: 42.5 s


In [ ]:
data

,ds,sales,ad_spend,temperature,unique_id
0,2025-07-15 21:20:06.526710,93.146283,51.357096,18.636706,sales
1,2025-07-16 21:20:06.526710,100.999198,59.402531,20.087807,sales
2,2025-07-17 21:20:06.526710,112.171016,53.719293,23.260143,sales
3,2025-07-18 21:20:06.526710,121.528659,63.837406,22.483753,sales
4,2025-07-19 21:20:06.526710,123.346967,71.372116,19.675663,sales
...,...,...,...,...,...
360,2026-07-10 21:20:06.526710,3702.016875,50.512889,21.826421,sales
361,2026-07-11 21:20:06.526710,3716.432589,60.695641,21.544221,sales
362,2026-07-12 21:20:06.526710,3716.909891,53.254844,18.317274,sales
363,2026-07-13 21:20:06.526710,3738.117654,57.458165,19.107960,sales


In [ ]:
forecast_df

,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,sales,2026-07-15 21:20:06.526710,3743.759277,3731.829346,3737.587646,3739.163086,3741.256592,3743.759277,3745.826660,3748.261475,3751.385254,3755.751221
1,sales,2026-07-16 21:20:06.526710,3757.420410,3746.345215,3748.969971,3754.191650,3755.510498,3757.420410,3762.216064,3762.026611,3767.710205,3768.325684
2,sales,2026-07-17 21:20:06.526710,3771.141357,3757.982910,3762.343018,3764.887695,3768.286621,3771.141357,3773.257324,3776.756104,3779.802490,3781.714111
3,sales,2026-07-18 21:20:06.526710,3776.363525,3768.265381,3768.935791,3771.207275,3773.729980,3776.363525,3781.160645,3782.773438,3786.900635,3789.259521
4,sales,2026-07-19 21:20:06.526710,3788.970703,3777.945801,3781.905762,3782.391113,3789.245117,3788.970703,3792.244873,3795.479004,3798.052734,3801.995117
...,...,...,...,...,...,...,...,...,...,...,...,...
123,sales,2026-11-15 21:20:06.526710,4970.268555,4693.992676,4841.908691,4918.560547,4957.769043,4970.268555,4977.485352,4983.155273,4991.789062,5010.577637
124,sales,2026-11-16 21:20:06.526710,4982.781250,4704.033203,4851.961914,4931.333496,4970.837891,4982.781250,4988.677246,4993.720703,5004.692383,5024.189941
125,sales,2026-11-17 21:20:06.526710,4990.425781,4712.064941,4861.351074,4939.832520,4980.495605,4990.425781,4997.383301,5004.020508,5012.360352,5032.639160
126,sales,2026-11-18 21:20:06.526710,5001.921387,4720.297852,4869.591797,4949.976562,4990.851562,5001.921387,5008.341309,5015.222168,5023.704102,5045.118652
